In [14]:
from loaddata import *
import pandas as pd
from random import random
from random import randint
from collections import Counter
import time

In [15]:
df = load_data('datasets/capture20110811.pcap.netflow.labeled')
df.head()

,Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Label,src_ip,src_port,dst_ip,dst_port
0,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,135,1,Background,89.31.8.11,23929,147.32.84.229,13363
1,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,276,1,Background,147.32.84.229,13363,89.31.8.11,23929
2,2011-08-11,10:10:00.006,0.000,UDP,INT,0,1,62,1,Background,208.88.186.6,34042,147.32.84.229,13363
3,2011-08-11,10:10:00.008,0.000,UDP,INT,0,1,78,1,Background,92.118.218.77,55246,147.32.84.229,13363
4,2011-08-11,10:10:00.009,0.000,UDP,INT,0,1,72,1,Background,182.185.139.181,10223,147.32.84.229,13363


In [16]:
# Get ground truth result
# Top10 most frequent IPs connected to the host
def get_top_connected_ip(seq):
    # Count results
    count = Counter(seq)
    result = count.most_common(10)
    return result


host_ip = '147.32.84.165'
seq = load_ip_sequence(df, host_ip)
truth = get_top_connected_ip(seq)
for str in truth:
    print(str)

('193.23.181.44', 6442)
('174.128.246.102', 4101)
('174.37.196.55', 3707)
('173.236.31.226', 3410)
('184.154.89.154', 3344)
('67.19.72.206', 3224)
('46.4.36.120', 3150)
('72.20.15.61', 3111)
('147.32.80.9', 1573)
('212.117.171.138', 967)


In [17]:
def reservoir_sampling(size, data):
    reservoir = data[0: reservoir_size]

    for i in range(size, len(data)):
        prob = size / i

        if random() >= prob:
            # Generate a random int to be the replaced sample index
            index = randint(0,size-1)
            # Replace old sample
            reservoir[index] = data[i]
    
    return reservoir 

In [19]:
# Get sampling result
# Try different reservoir size
for i in [100, 200, 500, 750, 1000, 2500, 5000, 7500, 10000, 20000, 30000, 50000]:
    print("********** size = %d **********" % i)
    reservoir_size = i
    
    # Reservior sampling
    start_time = time.time()
    result = reservoir_sampling(reservoir_size, seq)
    print("Build up time %.6f seconds ---" % (time.time() - start_time))
    
    # Count results
    count = Counter(result)
    print(count.most_common(10))

********** size = 100 **********
Build up time 0.102722 seconds ---
[('184.154.89.154', 12), ('217.163.21.39', 10), ('46.4.36.120', 7), ('83.133.119.197', 7), ('94.100.28.114', 6), ('212.117.171.138', 5), ('147.32.80.9', 4), ('98.139.175.225', 3), ('182.2.117.2', 3), ('98.137.54.237', 3)]
********** size = 200 **********
Build up time 0.079787 seconds ---
[('184.154.89.154', 28), ('217.163.21.39', 21), ('83.133.119.197', 9), ('98.139.175.225', 9), ('98.137.54.237', 9), ('94.100.28.114', 7), ('46.4.36.120', 7), ('212.117.171.138', 7), ('147.32.80.9', 6), ('184.82.147.252', 5)]
********** size = 500 **********
Build up time 0.077823 seconds ---
[('184.154.89.154', 82), ('46.4.36.120', 36), ('98.137.54.237', 25), ('217.163.21.39', 25), ('98.139.175.225', 19), ('94.100.28.114', 14), ('209.85.143.27', 14), ('147.32.80.9', 12), ('64.12.138.161', 11), ('83.133.119.197', 11)]
********** size = 750 **********
Build up time 0.086767 seconds ---
[('184.154.89.154', 100), ('46.4.36.120', 65), ('98